In [158]:
import numpy as np
import os
import pandas as pd

In [159]:
data = pd.read_csv("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/data_new.csv")

In [160]:
data

,Unnamed: 0,date,chemical,monitor,reading,wind_direction,wind_speed
0,0,2016-04-01 00:00:00,AGOC-3A,1,0.160462,190.5,4.0
1,1,2016-04-01 00:00:00,Appluimonia,1,0.003093,190.5,4.0
2,2,2016-04-01 00:00:00,Chlorodinine,1,0.382308,190.5,4.0
3,3,2016-04-01 00:00:00,Methylosmolene,1,0.417423,190.5,4.0
4,4,2016-04-01 00:00:00,AGOC-3A,2,0.228558,190.5,4.0
...,...,...,...,...,...,...,...
79483,79483,2016-12-31 23:00:00,Methylosmolene,8,0.539674,161.0,3.5
79484,79484,2016-12-31 23:00:00,AGOC-3A,9,0.830489,161.0,3.5
79485,79485,2016-12-31 23:00:00,Appluimonia,9,0.670133,161.0,3.5
79486,79486,2016-12-31 23:00:00,Chlorodinine,9,0.203557,161.0,3.5


In [183]:
def process(file_name):
    data = pd.read_csv(file_name)
    data.drop(data.columns[[0]], axis=1, inplace=True)
    data_new = pd.DataFrame()
    for chem_name in ["AGOC-3A", "Appluimonia", "Chlorodinine", "Methylosmolene"]:
        data_chem = data[data['chemical']==chem_name]
        data_chem = data_chem.reset_index(drop=True)
        data_chem["reading"].fillna(0, inplace=True)
        data_chem["cum_sum"] = data_chem["reading"].cumsum(axis = 0)
        count = 0
        random_data = data_chem["reading"]
        random_data_std = np.std(random_data)
        random_data_mean = np.mean(random_data)
        anomaly_cut_off = random_data_std * 3
        lower_limit  = random_data_mean - anomaly_cut_off 
        upper_limit = random_data_mean + anomaly_cut_off
        data_chem["anomaly_count"] = ''
        data_chem["dup_chem"] = ''
        o = 0
        for outlier in random_data:
            if outlier > upper_limit or outlier < lower_limit:
                data_chem["anomaly_count"][count] = 'a'
                o += 1
            count += 1
        print(o)
        dup = data_chem["reading"]
        for i in range(len(dup)-1):
            a = dup[i]
            b = dup[i+1]
#             c = dup[i+2]
#             if(a == b and b == c):
            if(a == b):
                print("dup_sensor")
                data_chem["dup_chem"][i] = 'd'
                data_chem["dup_chem"][i+1] = 'd'
#                 data_chem["dup_chem"][i+2] = 'd'
        data_new = data_new.append(data_chem)
    data_new["dup_all"] = ''
    date_unique = sorted(data_new["date"].unique())
    data_new['reading'] = data_new['reading'].replace(0, np.nan)
    for d in date_unique:
        data_dup = data_new[data_new['date']==d]
        data_dup = data_dup.reset_index(drop=True)
        data_new = data_new.reset_index(drop=True)
        if(len(data_dup) == 4 and not data_dup["reading"].nunique):
            print("dup_all")
            index = data_new.reset_index().merge(data_dup.reset_index(), on=['date','monitor','reading'])
            index = index.index_x
            for value in index:
                data_new["dup_all"][value] = 'd' 
    data_new['square_root'] = data_new['reading']**(1/2)
    data_new['cube'] = data_new['reading']**(3)
    return data_new

In [184]:
sensor_1 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_1.csv")
sensor_2 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_2.csv")
sensor_3 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_3.csv")
sensor_4 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_4.csv")
sensor_5 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_5.csv")
sensor_6 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_6.csv")
sensor_7 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_7.csv")
sensor_8 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_8.csv")
sensor_9 = process("/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_9.csv")
sensor_1.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_1_new.csv')
sensor_2.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_2_new.csv')
sensor_3.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_3_new.csv')
sensor_4.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_4_new.csv')
sensor_5.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_5_new.csv')
sensor_6.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_6_new.csv')
sensor_7.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_7_new.csv')
sensor_8.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_8_new.csv')
sensor_9.to_csv('/Users/raghav/Documents/DV/VAST-2017-MC2-Venkatramanan/MC2Data/sensor_9_new.csv')

/Users/raghav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/raghav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/raghav/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4
10
10
1
dup_sensor
7
17
5
3
16
36
36
16
28
3
29
13
26
36
32
11
41
11
59
23
8
52
23
10
6
10
8
2
33
33
38
dup_sensor
8
